In [1]:
import ROOT
from subprocess import call
from os.path import isfile

import warnings
warnings.filterwarnings('ignore')

Welcome to JupyROOT 6.18/00


In [2]:
def loaddata(name):
    filestring= '/home/waleed/forLewis/PID/files/dpm/dpmbkg_1M_' + name + '.root'
    file = ROOT.TFile.Open(filestring)
    particles = file.Get('t1')
    return file, particles

#load data
_ , electrons = loaddata('electrons')
_ , pions = loaddata('pions')
_ , muons = loaddata('muons')
_ , kaons = loaddata('kaons')
_ , protons = loaddata('protons')

In [3]:
# Add variables to dataloader
dataloader = ROOT.TMVA.DataLoader('dataset_pymva')
numVariables = len(electrons.GetListOfBranches())
for branch in electrons.GetListOfBranches():
    dataloader.AddVariable(branch.GetName())

# Add trees to dataloader
dataloader.AddTree(electrons, 'Electrons')
dataloader.AddTree(muons, 'Muons')
dataloader.PrepareTrainingAndTestTree(ROOT.TCut(''),'SplitMode=Random:NormMode=None:!V')

DataSetInfo              : [dataset_pymva] : Added class "Electrons"
                         : Add Tree t1 of type Electrons with 249846 events
DataSetInfo              : [dataset_pymva] : Added class "Muons"
                         : Add Tree t1 of type Muons with 249846 events
                         : Dataset[dataset_pymva] : Class index : 0  name : Electrons
                         : Dataset[dataset_pymva] : Class index : 1  name : Muons


In [4]:
# Setup TMVA
ROOT.TMVA.Tools.Instance()
ROOT.TMVA.PyMethodBase.PyInitialize()

outputFile = ROOT.TFile.Open('TMVAOutputPyMVA.root', 'RECREATE')
factory = ROOT.TMVA.Factory('TMVAClassification', outputFile,
        '!V:!Silent:Color:DrawProgressBar:Transformations=I,G:'+\
        'AnalysisType=Classification')

In [5]:
# Select Theano as backend for Keras
from os import environ
environ['KERAS_BACKEND'] = 'theano'

# Set architecture of system (AVX instruction set is not supported on SWAN)
environ['THEANO_FLAGS'] = 'gcc.cxxflags=-march=corei7'

from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.optimizers import Adam



# Define model
model = Sequential()
model.add(Dense(32, init='glorot_normal', activation='relu',
        input_dim=numVariables))
model.add(Dense(32, init='glorot_normal', activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(2, init='glorot_uniform', activation='softmax'))

# Set loss and optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(),
        metrics=['accuracy',])

# Store model to file
model.save('model.h5')

# Print summary of model
model.summary()

Using Theano backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                1952      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 3,074
Trainable params: 3,074
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Keras interface with previously defined model
factory.BookMethod(dataloader, ROOT.TMVA.Types.kPyKeras, 'PyKeras',
        'H:!V:VarTransform=G:FilenameModel=model.h5:'+\
        'NumEpochs=5:BatchSize=32:'+\
        'TriesEarlyStopping=3')

<ROOT.TMVA::MethodPyKeras object ("PyKeras") at 0x5559963f3620>

Factory                  : Booking method: PyKeras
                         : 
PyKeras                  : [dataset_pymva] : Create Transformation "G" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'Xmomentum' <---> Output : variable 'Xmomentum'
                         : Input : variable 'Ymomentum' <---> Output : variable 'Ymomentum'
                         : Input : variable 'Zmomentum' <---> Output : variable 'Zmomentum'
                         : Input : variable 'Momentum' <---> Output : variable 'Momentum'
                         : Input : variable 'Energy' <---> Output : variable 'Energy'
                         : Input : variable 'Theta' <---> Output : variable 'Theta'
                         : Input : variable 'Xposition' <---> Output : variable 'Xposition'
                         : Input : variable 'Yposition' <---> Output : variable 'Yposition'
       

In [7]:
# Run training, test and evaluation
factory.TrainAllMethods()

Train on 199877 samples, validate on 49969 samples
Epoch 1/5
199877/199877 [==============================] - 4s 18us/step - loss: 0.6962 - acc: 0.5017 - val_loss: 0.6932 - val_acc: 0.4989

Epoch 00001: val_loss improved from inf to 0.69323, saving model to dataset_pymva/weights/TrainedModel_PyKeras.h5
Epoch 2/5
199877/199877 [==============================] - 4s 20us/step - loss: 0.6932 - acc: 0.4991 - val_loss: 0.6931 - val_acc: 0.5011

Epoch 00002: val_loss improved from 0.69323 to 0.69314, saving model to dataset_pymva/weights/TrainedModel_PyKeras.h5
Epoch 3/5
199877/199877 [==============================] - 4s 22us/step - loss: 0.6932 - acc: 0.5003 - val_loss: 0.6932 - val_acc: 0.4989

Epoch 00003: val_loss did not improve from 0.69314
Epoch 4/5
199877/199877 [==============================] - 5s 24us/step - loss: 0.6932 - acc: 0.5004 - val_loss: 0.6931 - val_acc: 0.5011

Epoch 00004: val_loss did not improve from 0.69314
Epoch 5/5
199877/199877 [==============================] - 

Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth only supported for histograms with >= 3 bins. Nbins = 2
Error in <TH1F::Smooth>: Smooth 

In [ ]:
factory.TestAllMethods()
factory.EvaluateAllMethods()